In [34]:
import pandas as pd
from io import StringIO
import re
import textstat
import subprocess
from pydub import AudioSegment
from pydub.silence import detect_silence as silence
import requests
import os
import time
from urllib.request import urlretrieve
import tempfile
from nltk.tokenize import sent_tokenize as nltk_sent_tokenize
from deepmultilingualpunctuation import PunctuationModel
from transformers import pipeline
import warnings
import contextlib
import io
import sys
from datetime import datetime


subprocess.run(
    [sys.executable, '-c', 'from deepmultilingualpunctuation import PunctuationModel; PunctuationModel()'],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

warnings.filterwarnings("ignore", message="`grouped_entities` is deprecated")
warnings.filterwarnings("ignore", message="`Device set to use mps:0")

# Tokenizers
naive_sent_tokenize = lambda text: re.split(r'(?<=[.!?])\s+', text)
naive_word_tokenize = lambda text: re.findall(r'\b\w+\b', text)


def load_amplitude_data(pathway):
    with open(pathway, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    file_name = lines[0].strip().strip('"')
    csv_content = '\n'.join(lines[3:])

    # Load data
    df = pd.read_csv(StringIO(csv_content), delimiter='\t')

    # Clean column names
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(r'["\']', '', regex=True)
        .str.replace(r'[,\s]+$', '', regex=True)
    )

    # Fix trailing commas and quotes in numeric columns
    for col in df.columns:
        if df[col].dtype == 'object':
            if df[col].str.contains(r'^\s*\d+(?:\.\d+)?["\']?,\s*$', regex=True).any():
                df[col] = df[col].str.replace(r'["\']', '', regex=True)
                df[col] = df[col].str.rstrip(',')
                df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Strip trailing commas, quotes, and whitespace from string-like cells
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.strip().str.replace(r'["\',]+$', '', regex=True)

    return df

def combine_amplitude_data(df, new_pathway):
    with open(new_pathway, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    file_name = lines[0].strip().strip('"')
    csv_content = '\n'.join(lines[3:])

    # Load new data
    new_df = pd.read_csv(StringIO(csv_content), delimiter='\t')

    # Clean column names
    new_df.columns = (
        new_df.columns
        .str.strip()
        .str.replace(r'["\']', '', regex=True)
        .str.replace(r'[,\s]+$', '', regex=True)
    )

    # Clean and normalize CMS Urls in both dataframes
    df['CMS Url'] = df['CMS Url'].astype(str).str.strip()
    new_df['CMS Url'] = new_df['CMS Url'].astype(str).str.strip()

    # Fix numeric columns in new_df
    for col in new_df.columns:
        if new_df[col].dtype == 'object':
            new_df[col] = new_df[col].astype(str).str.strip().str.replace(r'["\',]+$', '', regex=True)
            if new_df[col].str.contains(r'^\s*\d+(?:\.\d+)?["\']?,?\s*$', regex=True).any():
                new_df[col] = pd.to_numeric(new_df[col], errors='coerce')

    # Only bring over the Watched Ads columns
    watched_ads_cols = [col for col in new_df.columns if 'Watched Ads' in col]
    columns_to_merge = ['CMS Url'] + watched_ads_cols

    # Merge on cleaned CMS Urls
    merged_df = df.merge(new_df[columns_to_merge], on='CMS Url', how='left')

    return merged_df

def extract_or_pass(val):
    hyperlink_pattern = r'HYPERLINK\("([^"]+)"'
    if isinstance(val, str):
        match = re.search(hyperlink_pattern, val)
        if match:
            return match.group(1)  # return the extracted URL
    return val  # return original if no match

def add_cms_data(df):
    # Extract UUID from CMS URL
    df['UUID'] = df['CMS Url'].str.extract(r'uuid=([a-f0-9\-]+)', expand=False).str.lower().str.strip()
    cms_df['UUID'] = cms_df['UUID'].astype(str).str.lower().str.strip()

    # Merge CMS data into main DataFrame
    merged_df = df.merge(cms_df[['UUID', 'Live Link', 'Publish Date']], on='UUID', how='left')

    # Convert Publish Date to datetime and extract components
    publish_datetime = pd.to_datetime(merged_df['Publish Date'], errors='coerce')
    merged_df['Publish Date'] = publish_datetime.dt.strftime('%Y-%m-%d')
    merged_df['Publish Hour'] = publish_datetime.dt.hour
    merged_df['Day of Week'] = publish_datetime.dt.day_name()
    merged_df['Month'] = publish_datetime.dt.month

    # Extract collection
    merged_df['Collection'] = merged_df['Live Link'].str.extract(r'https?://weather\.com/([^/]+)/', expand=False)

    # Convert necessary columns to numeric
    cols_to_convert = ['Videos Played--All Users', 'Watched Ads--All Users', 'Completion Rate--All Users']
    for col in cols_to_convert:
        if col in merged_df.columns:
            merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
        else:
            merged_df[col] = None

    # ✅ Compute monthly averages for normalization
    monthly_averages = (
        merged_df.groupby('Month')[cols_to_convert]
        .mean()
        .to_dict('index')
    )

    # ✅ Define scoring function INSIDE so it can see monthly_averages
    def calculate_content_score(row):
        month = row['Month']
        if month not in monthly_averages:
            return None

        avg = monthly_averages[month]

        try:
            norm_videos = row['Videos Played--All Users'] / avg['Videos Played--All Users']
            norm_ads = row['Watched Ads--All Users'] / avg['Watched Ads--All Users']
            norm_cr = row['Completion Rate--All Users'] / avg['Completion Rate--All Users']
        except ZeroDivisionError:
            return None

        # Weighted normalized score
        if month == 3:
            return norm_videos * 0.4047 + norm_ads * 0.3999 + norm_cr * 0.2
        elif month == 4:
            return norm_videos * 0.4009 + norm_ads * 0.3963 + norm_cr * 0.2074
        elif month == 5:
            return norm_videos * 0.3971 + norm_ads * 0.3928 + norm_cr * 0.2147
        elif month == 6:
            return norm_videos * 0.39331 + norm_ads * 0.3892 + norm_cr * 0.2221
        else:
            return None

    # Apply the scoring function
    merged_df['Content Score'] = merged_df.apply(calculate_content_score, axis=1)

    # Get rid of Month
    merged_df.drop(columns=['Month'], inplace=True)

    return merged_df

def add_video_metadata(df):
    global progress_bar

    df = df[df['Live Link'].notna() & df['Live Link'].str.startswith('http')]

    durations = []
    word_counts = []
    avg_sentence_length = []
    transcript_scores = []
    cut_counts = []
    avg_silences = []
    mp4_urls = []
    
    for idx, row in df.iterrows():
        url = row['Live Link']
        
        video_duration = None
        word_count = None
        avg_sentence_len = None
        readability = None
        cut_count = None
        avg_silence = None
        mp4_url = None

        try:
            resp = requests.get(url, timeout=10)
            html_text = resp.text
        except:
            pass

        # Duration
        match = re.search(r'"duration"\s*:\s*"((\d{1,2}:)?\d{1,2}:\d{2})"', html_text)
        if match:
            duration_str = match.group(1)
            time_parts = list(map(int, duration_str.split(":")))
        
            if len(time_parts) == 3:
                h, m, s = time_parts
            elif len(time_parts) == 2:
                h = 0
                m, s = time_parts
            elif len(time_parts) == 1:
                h, m, s = 0, 0, time_parts[0]
            else:
                h = m = s = 0
        
            video_duration = h * 3600 + m * 60 + s

        # Transcript
        transcript_match = re.search(r'"transcript"\s*:\s*"(.+?)"', html_text)
        if transcript_match:
            raw_transcript = transcript_match.group(1)

            # Convert raw transcript to punctuated transcript
            transcript = model.restore_punctuation(raw_transcript)
            
            # Tokenize and count
            words = naive_word_tokenize(transcript)
            sentences = naive_sent_tokenize(transcript)

            # Word Count and Average Sentence Length
            word_count = len(words)
            sentence_count = len(sentences)
            avg_sentence_len = word_count / sentence_count

            # Readability Scores
            readability = textstat.flesch_kincaid_grade(transcript)

        # MP4
        mp4_match = re.search(r'https?://[^"]+\.mp4', html_text)
        if mp4_match:
            mp4_url = mp4_match.group(0)
            mp4_urls.append(mp4_url)
        
            os.makedirs("audio_files", exist_ok=True)
            uuid = str(row['UUID']).strip()
            output_audio = os.path.join("audio_files", f"audio_{uuid}.wav")
        
            try:
                # Download mp4 to temp file
                with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as tmp_file:
                    urlretrieve(mp4_url, tmp_file.name)
                    local_mp4_path = tmp_file.name
        
                # Convert to wav for silence detection
                subprocess.run([
                    "ffmpeg", "-y", "-i", local_mp4_path,
                    "-vn", "-acodec", "pcm_s16le", "-ar", "44100", "-ac", "2",
                    output_audio
                ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        
                # Run ffprobe locally
                ffprobe_cmd = (
                    f'ffprobe -show_frames -of compact=p=0 '
                    + r'-f lavfi "movie=' + local_mp4_path + r',select=gt(scene\,0.3)"'
                )
        
                result = subprocess.run(
                    ffprobe_cmd, shell=True,
                    stdout=subprocess.PIPE, stderr=subprocess.DEVNULL,
                    text=True
                )
        
                scene_lines = result.stdout.strip().split('\n')
                scene_changes = [line for line in scene_lines if 'media_type=video' in line]
                cut_count = len(scene_changes)
        
            except Exception as e:
                pass
        
            try:
                audio = AudioSegment.from_file(output_audio, format="wav")
                silence_chunks = silence(
                    audio,
                    min_silence_len=100,
                    silence_thresh=audio.dBFS - 16
                )
                durations_list = [(end - start) / 1000 for start, end in silence_chunks]
                avg_silence = sum(durations_list) / len(durations_list) if durations_list else 0
            except Exception:
                pass
        
            finally:
                # Clean up temp mp4 file
                try:
                    os.remove(local_mp4_path)
                except Exception:
                    pass
        
        else:
            mp4_urls.append(None)

        durations.append(video_duration)
        word_counts.append(word_count)
        transcript_scores.append(readability)
        avg_sentence_length.append(avg_sentence_len)
        cut_counts.append(cut_count) 
        avg_silences.append(avg_silence)

        progress_bar += "█"
        print(f"\rProcessing videos: {progress_bar}", end="", flush=True)
    
    df = df.reset_index(drop=True)
    df['Video Length'] = durations
    df['Word Count'] = word_counts
    df['WPM'] = df['Word Count'] / (df['Video Length'] / 60)
    df['Readibility Score'] = transcript_scores
    df['Average Sentence Length'] = avg_sentence_length
    df['Number of Shots Changes'] = cut_counts
    df['Average Silence Length'] = avg_silences # average length of a silence chunk

    return df


start_time = time.time()

cms_df = pd.read_csv('/Users/parker.pape/Downloads/CMS Export - Sheet 1.csv')
amplitude_pathway = '/Users/parker.pape/Projects/Completion Rate Decline Analysis/Data Table - Comparing CR to Videos Played.csv'
new_pathway = '/Users/parker.pape/Projects/Completion Rate Decline Analysis/Data Table - Comparing CR to Watched Ads.csv'
output_pathway = '/Users/parker.pape/Projects/Completion Rate Decline Analysis/All Video Data.csv'

video_data = load_amplitude_data(amplitude_pathway) # Completion Rates and View Counts by Platform

video_data = combine_amplitude_data(video_data, new_pathway) # Ads Watched by Platform

video_data = add_cms_data(video_data) # Live Link, Publish Date, Publish Hour, Collection, Content Score

print("Processing videos:", end="", flush=True)
progress_bar = ""

video_data = add_video_metadata(video_data) # Video Length, Word Count, WPM, Readability Score, 
                                            # Avg Sentence Length, Number of Shots/Clips, Avg Silence Length

video_data_clean = video_data.dropna(axis=1, how='all')

# Output to csv
video_data_clean.to_csv(output_pathway, index=False)
print(f"\n\n✅ All video data saved to {output_pathway}")

# Calculate time and number of videos
elapsed_time = time.time() - start_time
print(f"✅ Elapsed Time: {elapsed_time:.2f} seconds")
print(f"Count: {len(video_data_clean)} videos")

Processing videos: ████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

✅ All video data saved to /Users/parker.pape/Projects/Completion Rate Decline Analysis/All Video Data.csv
✅ Elapsed Time: 5524.31 seconds
Count: 808 videos
